In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


<a href="https://colab.research.google.com/github/ialhashim/DenseDepth/blob/master/DenseDepth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/raajeshlr/DenseDepth.git

Cloning into 'DenseDepth'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 45 (delta 3), reused 45 (delta 3), pack-reused 0
Unpacking objects: 100% (45/45), done.


In [ ]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./DenseDepth/nyu.h5

--2020-07-11 17:46:09--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.108.187
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.108.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘./DenseDepth/nyu.h5’

./DenseDepth/nyu.h5 100%[===================>] 164.89M  12.0MB/s    in 16s     

2020-07-11 17:46:26 (10.3 MB/s) - ‘./DenseDepth/nyu.h5’ saved [172897376/172897376]



In [ ]:
#Manually make a copy of fg_bg.zip as depth_fg_bg.zip
#cp '/content/gdrive/My Drive/Colab Notebooks/EVA4/S15A/output/fg_bg.zip' '/content/depth_fg_bg.zip'

In [ ]:
!ls '/content/gdrive/My Drive/Colab Notebooks/EVA4/Session14/output/'

image  mask


In [ ]:
import zipfile

archive = zipfile.ZipFile('/content/gdrive/My Drive/Colab Notebooks/EVA4/Session14/fg_bg.zip')
for file in archive.namelist():
    if file.startswith('image/'):
        archive.extract(file, '/content/')

In [ ]:
!ls '/content/image' | wc -l

400000


In [ ]:
!mkdir '/content/depthmap'

In [ ]:
!ls '/content/depthmap' | wc -l

0


In [ ]:
#!rm -rf '/content/depthmap'

In [ ]:
!pwd

/content


In [ ]:
cd DenseDepth

/content/DenseDepth


In [ ]:
import sys
workingdir = '/content/DenseDepth'
sys.path.append(workingdir)

In [ ]:
!ls

augment.py	demo.py		  examples   model.py	Tensorflow
callbacks.py	demo_rgb.npy	  layers.py  nyu.h5	test.py
data.py		DenseDepth.ipynb  LICENSE    PyTorch	train.py
demo_depth.npy	evaluate.py	  loss.py    README.md	utils.py


In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import notebook
import zipfile

def load_images(path, start, end):
    loaded_images = []
    for i in range(start, end+1):
        x = np.clip(np.asarray(Image.open(f"{path}fg_bg_{str(i)}.jpg" ).resize((256, 256)), dtype=float) / 255, 0, 1)
        #x = np.clip(np.asarray(Image.open(f"{path}fg_bg_{str(i)}.jpg" ), dtype=float) / 255, 0, 1)
        loaded_images.append(x)
    return np.stack(loaded_images, axis=0)


def write_images(out_zip, path, start, outputs, inputs=None, gt=None, is_colormap=True, is_rescale=True, log=False):
    import matplotlib.pyplot as plt
    import skimage
    from skimage.transform import resize

    plasma = plt.get_cmap('gray')

    shape = (outputs[0].shape[0], outputs[0].shape[1], 3)
    if log:
        print(shape)
    
    #all_images = []
    start = start
    for i in notebook.tqdm(range(outputs.shape[0]),desc = f"Loading from {start} images") :
    
        if is_colormap:
            rescaled = outputs[i][:,:,0]
            if is_rescale:
                rescaled = rescaled - np.min(rescaled)
                rescaled = rescaled / np.max(rescaled)
  
            plt.figure(figsize=(2.24,2.24),dpi=100)
            matplotlib_image = plt.imshow(plasma(rescaled)[:,:,:3])
            pil_image = Image.fromarray(np.uint8( ( matplotlib_image.get_array()*255))).convert("L").resize((220, 220))
            #pil_image = Image.fromarray(np.uint8( ( matplotlib_image.get_array()*255))).convert("L")
            pil_image.save('imgtemp.jpg', optimize=True, quality=30)
            out_zip.write('imgtemp.jpg',f'depthmap/depth_{str(start)}.jpg')
            #if log:
            #    print("Saving Image: ", f"{path}depth_{str(start)}.jpg")
            #pil_image.save(f"{path}depth_{str(start)}.jpg")
            plt.close()
            pil_image = None 
            matplotlib_image = None
            start+=1

In [ ]:
#!ls '/content/image'

In [ ]:
import test
import time
import gc


INPATH = "/content/image/"
OUTPATH = "/content/depthmap/"

start_time = time.time()

#total_images = 400000
#total_images = 4000
batch_size = 500
#num_iter = int(total_images/batch_size)
num_iter = 55
image_start = 56501
run = 3

#print("Generating Depth Images for:", total_images, "in batches of:", batch_size, "with ", num_iter, "iterations")

model = test.loadmodel()

outputs = []
inputs = []
log = False

for  i in range(1, num_iter+1):
  iter_start = time.time()
  out_zip =zipfile.ZipFile(f'/content/gdrive/My Drive/Colab Notebooks/EVA4/Session14/output/depth_{str(run)}.zip', mode='a', compression=zipfile.ZIP_STORED)
  start = image_start
  end = image_start + batch_size - 1
  print("Generating Image: ", image_start, " to ", end)
  inputs = load_images(INPATH, start, end)
  print('\nLoaded ({0}) images of size {1}.'.format(inputs.shape[0], inputs.shape[1:]))
  outputs = test.predict_output(model, inputs, log=True)
  write_images(out_zip, OUTPATH, start, outputs.copy(), log=True)
  print("Iteration:", i, "--- %s seconds ---" % (time.time() - iter_start), " --- Total : %s seconds ---" % (time.time() - start_time))
  image_start = image_start + batch_size
  inputs = None
  outputs = None
  out_zip.close()
  gc.collect()

print("--- %s seconds ---" % (time.time() - start_time))


Loading model...
Generating Image:  56501  to  57000

Loaded (500) images of size (256, 256, 3).
(128, 128, 3)



Iteration: 1 --- 65.18450331687927 seconds ---  --- Total : 76.10345101356506 seconds ---
Generating Image:  57001  to  57500

Loaded (500) images of size (256, 256, 3).
(128, 128, 3)



Iteration: 2 --- 43.25440073013306 seconds ---  --- Total : 120.2833616733551 seconds ---
Generating Image:  57501  to  58000

Loaded (500) images of size (256, 256, 3).
(128, 128, 3)



Iteration: 3 --- 42.917328119277954 seconds ---  --- Total : 164.50643038749695 seconds ---
Generating Image:  58001  to  58500

Loaded (500) images of size (256, 256, 3).
(128, 128, 3)



Iteration: 4 --- 40.37646985054016 seconds ---  --- Total : 206.61127710342407 seconds ---
Generating Image:  58501  to  59000

Loaded (500) images of size (256, 256, 3).
(128, 128, 3)


...


Iteration: 51 --- 42.3550500869751 seconds ---  --- Total : 2728.044055223465 seconds ---
Generating Image:  82001  to  82500

Loaded (500) images of size (256, 256, 3).
(128, 128, 3)



Iteration: 52 --- 44.71403932571411 seconds ---  --- Total : 2793.6635320186615 seconds ---
Generating Image:  82501  to  83000

Loaded (500) images of size (256, 256, 3).
(128, 128, 3)



Iteration: 53 --- 42.339516401290894 seconds ---  --- Total : 2857.700156211853 seconds ---
Generating Image:  83001  to  83500

Loaded (500) images of size (256, 256, 3).
(128, 128, 3)



Iteration: 54 --- 42.34999203681946 seconds ---  --- Total : 2921.937208890915 seconds ---
Generating Image:  83501  to  84000

Loaded (500) images of size (256, 256, 3).
(128, 128, 3)



Iteration: 55 --- 45.36383509635925 seconds ---  --- Total : 2989.5933105945587 seconds ---
--- 3012.4109785556793 seconds ---
